In [18]:
import numpy as np
import sys
sys.path.append(r"C:\Users\gapar\OneDrive\Documents\GitHub\Logical-Clifford-Synthesis")
from algorithm_1 import SymplecticOperations
%load_ext autoreload
%autoreload 2


# Initialize the SymplecticOperations class
symplectic_ops = SymplecticOperations()

# Example usage


F, Transvecs = symplectic_ops.find_symp_mat_transvecs(X, Y)

# Use F and Transvecs as needed


def symplectic_code(U, V):

    m = U.shape[1] // 2
    k = 2 * m - V.shape[0]
    tot = 2 ** (k * (k + 1) // 2)
    F_all = [None] * tot

    # Find one solution using symplectic transvections
    F0, Transvecs = symplectic_ops.find_symp_mat_transvecs(U[0:(2*m-k), :], V)
    #F0 = find_symp_mat_transvecs(U[0:(2*m-k), :], V)

    A = U.dot(F0) % 2
    Ainv = gf2matinv(A)
    Basis = np.vstack((A[(m - k + 1):m, :], A[(2*m - k + 1):(2*m), :]))
    Subspace = np.dot(np.unpackbits(np.arange(2 ** (2 * k))[:, None], axis=1), Basis) % 2
    StabF0 = A[(m - k + 1):m, :]
    Choices = [None] * k

    # Calculate all choices for each vector in Subspace using just the conditions
    # imposed by the (modified) stabilizer generators, i.e., the rows of S*F0
    for i in range(k):
        # Impose symplectic inner product of 1 with the "fixed" stabilizer
        h = np.concatenate((np.zeros(i, dtype=int), [1], np.zeros(k - i, dtype=int)))
        Innpdts = np.dot(Subspace, np.fft.fftshift(StabF0, axes=1).T) % 2
        Choices[i] = Subspace[np.all(Innpdts == h, axis=1)]

    # First free vector, i.e. row 1 of Subspace has 2^k choices,
    # second free vector has 2^(k-1) choices and so on...
    for l in range(tot):
        Bl = np.copy(A)
        V = np.zeros((k, 2 * m), dtype=int)
        lbin = np.unpackbits(np.array([l], dtype=np.uint64).view(np.uint8))[::-1][:k * (k + 1) // 2]
        v1_ind = np.packbits(lbin[:k], bitorder='big')[0]
        V[0, :] = Choices[0][v1_ind]
        for i in range(1, k):
            vi_ind = np.packbits(lbin[k*(i-1):k*i], bitorder='big')[0]
            Innprods = np.dot(Choices[i], np.fft.fftshift(V, axes=1).T) % 2
            
            # Impose symplectic inner product of 0 with chosen free vectors
            Ch_i = Choices[i][np.all(Innprods == 0, axis=1)]
            V[i, :] = Ch_i[vi_ind]
        Bl[(2 * m - k):(2 * m), :] = V
        F = np.dot(Ainv, Bl) % 2
        F_all[l] = np.dot(F0, F) % 2

    return F_all


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
X = np.array([[1, 0], [0, 1]])
Y = np.array([[0,1], [1, 1]])
symplectic_code(X,Y)



NameError: name 'gf2matinv' is not defined